## Importing files

In [43]:
import keras
from keras.datasets import cifar10
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input
import scipy
from scipy import misc
import os

## Load data

In [44]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
y_train = np.squeeze(y_train)

## Load the model

In [45]:
model = VGG16(weights='imagenet',include_top = False, input_shape = (64,64,3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

## Obtain bottleneck features for train dataset

In [46]:
if os.path.exists('vgg16_features_train.npz'):
    print('bottleneck features detected train')
    features = np.load('vgg16_features_train.npz')['features']
else:
    print('bottleneck features are calculated now')
    big_x_train = np.array([scipy.misc.imresize(x_train[i], (64, 64, 3)) 
                            for i in range(0, len(x_train))]).astype('float32')
    vgg16_input_train = preprocess_input(big_x_train)
    print('train data preprocessed')
    features = model.predict(vgg16_input_train)
    np.savez('vgg16_features_train',features = features)
print('bottleneck features saved for train')

bottleneck features detected train
bottleneck features saved for train


## Obtain bottleneck features for test dataset

In [47]:
if os.path.exists('vgg16_features_test.npz'):
    print('bottleneck features detected test')
    features_test = np.load('vgg16_features_test.npz')['features_test']
else:
    print('bottleneck features are calculated now')
    big_x_test = np.array([scipy.misc.imresize(x_test[i], (64, 64, 3)) 
                       for i in range(0, len(x_test))]).astype('float32')
    vgg16_input_test = preprocess_input(big_x_test)
    print('test data preprocessed')
    features_test = model.predict(vgg16_input_test)
    np.savez('vgg16_features_test',features_test = features_test)
print('bottleneck features saved for test')

bottleneck features detected test
bottleneck features saved for test


## One hot encoding the labels

In [48]:
for i in range(5):
    print(y_train[i])

6
9
9
4
1


In [49]:
from keras.utils import to_categorical
number_of_labels = len(np.unique(y_train))
y_train = to_categorical(y_train,num_classes=number_of_labels)
y_test = to_categorical(y_test,num_classes=number_of_labels)
for i in range(5):
    print(y_train[i])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


## Model Architecture

In [50]:
from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential
model = Sequential()
model.add(Flatten(input_shape=(2,2,512)))
model.add(Dense(500, activation= 'relu'))
model.add(Dropout(0.5))
model.add(Dense(500, activation= 'relu'))
model.add(Dropout(0.15))
model.add(Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 500)               1024500   
_________________________________________________________________
dropout_5 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 500)               250500    
_________________________________________________________________
dropout_6 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5010      
Total params: 1,280,010
Trainable params: 1,280,010
Non-trainable params: 0
_________________________________________________________________


## Compile the model

In [51]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

## Train the model

In [52]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath = 'weights_Transfer_learning.best.hdf5', verbose = 1, save_best_only = True)
model.fit(features,y_train, epochs=10, batch_size=100, verbose=1,callbacks=[checkpoint], validation_split=0.15, shuffle=True)

Train on 42500 samples, validate on 7500 samples
Epoch 1/10
42500/42500 [==============================] - 22s 529us/step - loss: 2.3126 - acc: 0.6091 - val_loss: 0.8426 - val_acc: 0.7239

Epoch 00001: val_loss improved from inf to 0.84261, saving model to weights_Transfer_learning.best.hdf5
Epoch 2/10
42500/42500 [==============================] - 23s 543us/step - loss: 0.9382 - acc: 0.7179 - val_loss: 0.7773 - val_acc: 0.7495

Epoch 00002: val_loss improved from 0.84261 to 0.77731, saving model to weights_Transfer_learning.best.hdf5
Epoch 3/10
42500/42500 [==============================] - 27s 642us/step - loss: 0.8461 - acc: 0.7421 - val_loss: 0.7354 - val_acc: 0.7616

Epoch 00003: val_loss improved from 0.77731 to 0.73536, saving model to weights_Transfer_learning.best.hdf5
Epoch 4/10
42500/42500 [==============================] - 25s 596us/step - loss: 0.7988 - acc: 0.7636 - val_loss: 0.7396 - val_acc: 0.7661

Epoch 00004: val_loss did not improve from 0.73536
Epoch 5/10
42500/425

## Load the model weights that yielded best validation accuracy

In [53]:
model.load_weights('weights_Transfer_learning.best.hdf5')

## Evaluate model classification accuracy on test set

In [54]:
score= model.evaluate(features_test,y_test,verbose=0)
print(model.metrics_names)
print(score)
accuracy=100*score[1]

['loss', 'acc']
[0.7044107089042664, 0.7872]


In [55]:
print('Test accuracy:',accuracy)

Test accuracy: 78.72
